In [1]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install cirq --quiet
    import cirq
    print("installed cirq.")

installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.6 MB/s eta 0:00:00
     ━━━

In [21]:
import cirq
import matplotlib.pyplot as plt

def less_than_k(k, list_n):
    # Number of qubits needed based on k
    num_qubits = len(bin(k)) - 2

    # Create quantum registers and quantum circuit
    qr = cirq.LineQubit.range(num_qubits)
    circuit = cirq.Circuit()

    # Apply Hadamard gates to create superposition
    circuit.append(cirq.H.on_each(*qr))

    # Apply Grover's algorithm
    num_iterations = int((len(list_n) ** 0.5) * 0.5)  # Approximately optimal number of iterations
    for _ in range(num_iterations):
        # Apply Oracle marking elements less than k
        oracle = create_oracle(k, list_n, num_qubits)
        circuit.append(oracle)

        # Apply Grover diffusion operator
        circuit.append(grover_diffusion(qr))

    # Measure qubits
    circuit.append(cirq.measure(*qr, key='result'))

    # Print quantum circuit
    print("Quantum Circuit:")
    print(circuit)
    print()

    # Simulate the circuit
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=10)  # Increased repetitions for better probability estimation

    # Extract the result
    result_str = result.measurements['result']

    # Interpret result bitstring as a number less than k
    output = []
    for bits in result_str:
        number = int(''.join(str(bit) for bit in bits), 2)
        if number < k and number in list_n:
            output.append(number)

    return output

def create_oracle(k, list_n, num_qubits):
    # Define oracle circuit
    oracle = cirq.Circuit()

    # Create control qubits
    control_qubits = cirq.LineQubit.range(num_qubits)
    target_qubit = cirq.LineQubit(num_qubits)

    # Apply phase flip on target qubit if the number represented by the state of control qubits is less than k
    for i, qubit in enumerate(control_qubits):
        # Check if the number represented by the state of control qubits is less than k
        if i < len(list_n) and list_n[i] < k:
            oracle.append(cirq.X(qubit))

    # Apply multi-controlled Z gate
    oracle.append(cirq.X(target_qubit).controlled_by(*control_qubits))

    return oracle

def grover_diffusion(qubits):
    # Define diffusion circuit
    num_qubits = len(qubits)
    diffusion = cirq.Circuit()

    # Apply Hadamard gates
    diffusion.append(cirq.H.on_each(*qubits))

    # Apply X gates
    diffusion.append(cirq.X.on_each(*qubits))

    # Apply multi-controlled Z gate
    control_qubits = qubits[:-1]  # All qubits except the last one
    target_qubit = qubits[-1]
    diffusion.append(cirq.Z(target_qubit).controlled_by(*control_qubits))

    # Apply X gates
    diffusion.append(cirq.X.on_each(*qubits))

    # Apply Hadamard gates
    diffusion.append(cirq.H.on_each(*qubits))

    return diffusion

# Example usage
A = less_than_k(7, [4, 9, 11, 14, 1, 13, 6, 15])
print("Output:", A)  # Output: [4, 1, 6]


Quantum Circuit:
0: ───H───X───@───H───X───@───X───H───M('result')───
              │           │           │
1: ───H───────@───H───X───@───X───H───M─────────────
              │           │           │
2: ───H───────@───H───X───@───X───H───M─────────────
              │
3: ───────────X─────────────────────────────────────

Output: [1, 6, 4]
